<a href="https://colab.research.google.com/github/Taiser-Riad/Titanic-Survival-Prediction/blob/main/NN_Pre_trained_Taiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


Taiseroooo

In [ ]:
!pip install unidecode
import datasets
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
import warnings
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
dataset = datasets.load_dataset("Nicolas-BZRD/Parallel_Global_Voices_English_French", split="train")
print(dataset.shape)
import torch
print("ماشي طيب")

(342060, 2)
ماشي طيب


In [ ]:
!pip install transformers

In [ ]:
!pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer
import datasets

# Load the dataset
dataset = datasets.load_dataset("Nicolas-BZRD/Parallel_Global_Voices_English_French", split="train")

# Split the dataset into training and test sets using Hugging Face's method
split_datasets = dataset.train_test_split(test_size=0.1, seed=12)
train_dataset = split_datasets['train']
test_dataset = split_datasets['test']

# Load pre-trained model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-fr'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Function to translate text from English to French
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
for i in range(50):  # Iterate over the first 5 sentences in the test dataset
    english_text = test_dataset[i]['en']  # Directly access the 'en' field
    french_translation = translate(english_text)
    print(f"English: {english_text}")
    print(f"French: {french_translation}\n")


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

English: Samuel Bran reflects on poverty [es] in his country:
French: Samuel Bran se penche sur la pauvreté dans son pays :

English: But it is grand is to see all those happy faces of the children.
French: Mais il est grand de voir tous ces visages heureux des enfants.

English: On Thursday 25 March 2010, people in hundreds of cities around the world including Accra will come together offline to rally around the important cause of Education by hosting local events to have fun and create awareness.
French: Jeudi 25 mars 2010, des personnes dans des centaines de villes du monde entier, dont Accra, se réuniront hors ligne pour se rassembler autour de l'importante cause de l'Education en accueillant des événements locaux pour s'amuser et créer une prise de conscience.

English: I suspected that the real goal was to create martyrs in an attempt to sway the middle class in Bangkok to their side.
French: Je soupçonnais que le but réel était de créer des martyrs dans une tentative d'écarter l

In [ ]:
from sacrebleu.metrics import BLEU

def calculate_bleu_score(model, tokenizer, dataset, num_samples=100):
    bleu = BLEU()
    references = []
    hypotheses = []

    for i in range(num_samples):
        # Extract the English and French texts
        english_text = dataset['en'][i]
        french_text = dataset['fr'][i]

        # Translate the English text
        translated_text = translate(english_text)

        # Store the references and hypotheses
        references.append([french_text])  # Reference translations are expected to be a list of lists
        hypotheses.append(translated_text)

    # Calculate the BLEU score
    score = bleu.corpus_score(hypotheses, references)
    return score

# Example usage
bleu_score = calculate_bleu_score(model, tokenizer, test_dataset, num_samples=100)
print(f"BLEU score: {bleu_score.score}")


BLEU score: 12.192091596713041


In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer
import datasets
from sacrebleu.metrics import BLEU
import re
import string

# Function for data preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Load and preprocess the dataset
def preprocess_dataset(dataset):
    return dataset.map(lambda x: {'en': preprocess_text(x['en']), 'fr': preprocess_text(x['fr'])})

dataset = datasets.load_dataset("Nicolas-BZRD/Parallel_Global_Voices_English_French", split="train")
dataset = preprocess_dataset(dataset)

# Split the dataset into training and test sets
split_datasets = dataset.train_test_split(test_size=0.1, seed=12)
train_dataset = split_datasets['train']
test_dataset = split_datasets['test']

# Load pre-trained model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-fr'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Function to translate text from English to French
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Function to calculate BLEU score
def calculate_bleu_score(model, tokenizer, dataset, num_samples=100):
    bleu = BLEU()
    references = []
    hypotheses = []

    for i in range(num_samples):
        english_text = dataset['en'][i]
        french_text = dataset['fr'][i]
        translated_text = translate(english_text)
        references.append([french_text])
        hypotheses.append(translated_text)

    score = bleu.corpus_score(hypotheses, references)
    return score

# Evaluate the model
bleu_score = calculate_bleu_score(model, tokenizer, test_dataset, num_samples=100)
print(f"BLEU score: {bleu_score.score}")


Map:   0%|          | 0/342060 [00:00<?, ? examples/s]

BLEU score: 13.134549472120794


In [ ]:
from ipywidgets import widgets, interact

def interactive_translate(text):
    translated_text = translate(text)
    print(f"Translated Text: {translated_text}")

interact(interactive_translate, text=widgets.Text(value='Enter text here', description='English:'));
